In [1]:
using Flux
using Metal
using CSV
using DataFrames
using OneHotArrays
using Statistics


In [2]:
Metal.functional()
device = Flux.get_device(; verbose=true)
device.deviceID


[ Info: Using backend: Metal.


<AGXG14GDevice: 0x12b281a00>
    name = Apple M2

In [3]:
# load breast cancer dataset
# csv_file_path = metal_dir * "data/breast-cancer-wisconsin.data"
csv_file_path = "data/breast-cancer-wisconsin.data"

"data/breast-cancer-wisconsin.data"

In [4]:
df_orig = CSV.File(csv_file_path) |> DataFrame;
column_names = [:SampleCodeNumber, :ClumpThickness, :UniformityOfCellSize,
:UniformityOfCellShape, :MarginalAdhesion, :SingleEpithelialCellSize,
:BareNuclei, :BlandChromatin, :NormalNucleoli, :Mitoses, :Class];
rename!(df_orig, column_names);

In [5]:
# # Remove unwanted columns
df = df_orig[:, Not(Cols(:BareNuclei, :SampleCodeNumber))];

In [6]:
# Define a dictionary to map values
class_mapping = Dict(2 => 0, 4 => 1)
# Use the map function to apply the mapping
df[!, :Class] = map(
    x -> class_mapping[x], 
    df[:, :Class]
);

In [7]:
model = Chain(
    Dense(8 => 2),  # model with 8 features and 2 classes
    σ               # sigmoid activation
) |> device         # output model to GPU

Chain(
  Dense(8 => 2),                        # 18 parameters
  NNlib.σ,
) 

In [8]:
classes = [0, 1]
accuracy(x, y) = mean(onecold(model(x), classes) .== y);
# accuracy(x, y_orig)

In [9]:
# function loss(ŷ, y)
#     Flux.logitcrossentropy(ŷ, y)
# end;

function loss(ŷ, y)
    Flux.binarycrossentropy(ŷ, y)
end;

In [10]:
X_train = Matrix{Float64}(df[:, Not(:Class)])
X_train = transpose(X_train)
y_train = vec(df[:, :Class]);

In [11]:
optimizer = Flux.setup(Adam(), model)
train_loader = Flux.DataLoader((X_train, y_train), batchsize=1, shuffle=false)

698-element DataLoader(::Tuple{LinearAlgebra.Transpose{Float64, Matrix{Float64}}, Vector{Int64}})
  with first element:
  (8×1 Matrix{Float64}, 1-element Vector{Int64},)

In [12]:
epochs = 1;

In [13]:
for epoch in 1:epochs
    total_accuracy = 0.
    total_loss = 0.
    for (x_cpu, y_cpu) in train_loader
        # pass the input and label to the GPU
        input = reshape(x_cpu, :) |> device
        label = y_cpu[1] |> device
        # calculate the gradient
        ∂f∂x = gradient(m -> loss(m(input), label), model)
        Flux.update!(optimizer, model, ∂f∂x[1])
        # calculate accuracy and loss
        ŷ = model(input)
        total_accuracy += accuracy(input, label)
        total_loss += loss(ŷ, label)
    end
    s = length(train_loader)
    avg_accuracy = total_accuracy / s
    avg_loss = total_loss / s
    println("Epoch $epoch: Accuracy=$avg_accuracy, Loss=$avg_loss")
end

Epoch 1: Accuracy=0.673352435530086, Loss=2.077824599405434


<img src=images/fluxnn.png width='50%' height='50%' > </img>